In [1]:
require 'nn'
require 'rnn'

In [2]:
-- Some useful functions
function genNbyK(n, k, a, b)
    out = torch.LongTensor(n, k)
    for i=1, n do
        for j = 1, k do
            out[i][j] = torch.random(a, b)
        end
    end
    return out
end

function buildModel(model, vocabSize, embeddingSize, metric, adapt, use_cuda)
    -- Small experiments seem to show that the Tanh activations performed better\
    --      than the ReLU for the bow model
    if model == 'bow' then
        print(string.format("Running bag-of-words model to learn %s", metric))
        sentenceLookup = nn.Sequential()
                    :add(nn.LookupTableMaskZero(vocabSize, embeddingSize))
                    :add(nn.Sum(2, 3, true)) -- Not averaging blows up model so keep this true
                    :add(nn.Tanh())
    else
        print(string.format("Running LSTM model to learn %s", metric))
        sentenceLookup = nn.Sequential()
                    :add(nn.LookupTableMaskZero(vocabSize, embeddingSize))
                    :add(nn.SplitTable(2))
                    :add(nn.Sequencer(nn.LSTM(embeddingSize, embeddingSize)))
                    :add(nn.SelectTable(-1))            -- selects last state of the LSTM
                    :add(nn.Linear(embeddingSize, embeddingSize))
                    :add(nn.ReLU())
    end
    local queryLookup = sentenceLookup:clone("weight", "gradWeight") 
    local summaryLookup = sentenceLookup:clone("weight", "gradWeight")
    local pmodule = nn.ParallelTable()
                :add(sentenceLookup)
                :add(queryLookup)
                :add(summaryLookup)

    if model == 'bow' then
        nnmodel = nn.Sequential()
            :add(pmodule)
            :add(nn.JoinTable(2))
            :add(nn.Tanh())
            :add(nn.Linear(embeddingSize * 3, 2))
    else
        nnmodel = nn.Sequential()
            :add(pmodule)
            :add(nn.JoinTable(2))
            :add(nn.ReLU())
            :add(nn.Linear(embeddingSize * 3, 2))
    end

    if adapt then 
        print("Adaptive regularization")
        local logmod = nn.Sequential()
            :add(nn.Linear(embeddingSize * 3, 1))
            :add(nn.LogSigmoid())
            :add(nn.SoftMax())

        local regmod = nn.Sequential()
            :add(nn.Linear(embeddingSize * 3, 2))

        local fullmod = nn.ConcatTable()
            :add(regmod)
            :add(logmod)

        local final = nn.Sequential()
            :add(pmodule)
            :add(nn.JoinTable(2))
            :add(fullmod)

        nnmodel = final
    end

    if use_cuda then
        return nnmodel:cuda()
    end
    return nnmodel
end

function buildPredsummary(summary, chosenactions, inputsentences, select_index)
    if summary == nil then
        summary = torch.zeros(inputsentences:size())
    end
    for i=1, chosenactions:size(1) do
        -- the 2 is for the SELECT index, will have to make this more general later
        if chosenactions[i][select_index] == 1 then
            summary[i]:copy(inputsentences[i])
        end
    end    
    return summary
end

function buildPredsummaryFast(summary, chosenactions, inputsentences, select_index)
    n = inputsentences:size(1)
    k = inputsentences:size(2)
    if summary == nil then
        summary = torch.zeros(inputsentences:size())
    end
    actionmatrix = chosenactions:select(2, select_index):clone():resize(n, 1):view(n, 1):expand(n, k):clone()
--     actionmatrix = chosenactions:select(2, select_index):resize(1, n):view(n, 1):expand(n, k):clone()
    return actionmatrix:cmul(inputsentences:double())
end

In [3]:
-- Setting parameters
n = 10
n_s = 5
k = 7
q = 5
a = 1
b = 100
embDim = 50
SKIP = 1
SELECT = 2

In [4]:
-- Simulating streams and queries
queries = genNbyK(n, q, a, b)

-- Note that the sentences are batched by sentence index so sentences[1] is the first sentence of each article
sentences = {}
for i=1, n_s do
    sentences[i] = genNbyK(n, k, a, b)
end

-- Using this to generate the optimal actions
true_actions = {}
for i=1, n_s do
    ---- Simulating the data
    trueqValues = torch.rand(n, 2)
    
     ---- Generating the max values and getting the indices
    qMaxtrue, qindxtrue = torch.max(trueqValues, 2)
    
    --- I want to select the qindx elements for each row
    true_actions[i] = torch.zeros(n, 2):scatter(2, qindxtrue, torch.ones(trueqValues:size()))
end

In [5]:
local SKIP = 1
local SELECT = 2

-- Simulating streams and queries
queries = genNbyK(n, q, a, b)

-- Note that the sentences are batched by sentence index so sentences[1] is the first sentence of each article
sentences = {}
for i=1, n_s do
    sentences[i] = genNbyK(n, k, a, b)
end

-- Using this to generate the optimal actions
true_actions = {}
for i=1, n_s do 
    ---- Simulating the data
    trueqValues = torch.rand(n, 2)
    
     ---- Generating the max values and getting the indices
    qMaxtrue, qindxtrue = torch.max(trueqValues, 2)
    
    --- I want to select the qindx elements for each row
    true_actions[i] = torch.zeros(n, 2):scatter(2, qindxtrue, torch.ones(trueqValues:size()))
end

model = buildModel('bow', b, embDim, 'f1', false, false)

Running bag-of-words model to learn f1	


In [6]:
function buildTotalSummary(predsummary, totalPredsummary)
    nps = predsummary:size(1)
    n_l = totalPredsummary:size(2)
    indices = torch.linspace(1, n_l, n_l):long() 
    for i=1, predsummary:size(1) do
        if predsummary[i]:sum() > 0 then 
            maxindex = 0
            for j = 1, totalPredsummary[i]:size(1) do 
                if totalPredsummary[i][j] == 0 then
                    maxindex = maxindex + 1
                end
            end
            lenx = predsummary[i]:size(1)
            totalPredsummary[i][{{maxindex - lenx + 1, maxindex}}]:copy(predsummary[i])
        end
    end
end

In [7]:
function buildTotalSummaryFast(predsummary, totalPredsummary)
    nps = predsummary:size(1)
    n_l = totalPredsummary:size(2)
    indices = torch.linspace(1, n_l, n_l):long() 
    for i=1, predsummary:size(1) do
        if predsummary[i]:sum() > 0 then 
            -- Finding the largest index with a zero
            maxindex = torch.max(indices[torch.eq(totalPredsummary[i], 0)])
            lenx = predsummary[i]:size(1)
            totalPredsummary[i][{{maxindex - lenx + 1, maxindex}}]:copy(predsummary[i])
        end
    end
end

In [11]:
totalPredsummary

Columns 1 to 16
   0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
   0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
   3   53  100   40   47   44   20   66   23   11   74   64   82   20   80   94
   0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
  60    8   10   68   57   48   14   91   93   31   47   74   79   91   14   82
   0    0    0    0    0    0    0   82   73   58   85   87   32   65   67   29
  22   65   37    9   20    2   81   10   18   42   25   93   30   85   26   66
   0    0    0    0    0    0    0   34   26   39   24   96   53   44   10   84
  54   19   64   26   63   19   11   46   23   44   65   80   15   35   73   37
  95   17   56   93   88   83   10   93    8   32   64   50    7   68   93   95

Columns 17 to 32
   0    0    0    0    0   75   22   65    4   77   24   59   35   42   67   39
   0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
  22  

In [10]:
sentences[n_s], predsummary

  69   65   34   19   54   93   78
  63   83   40   75   22   65    2
   3   53  100   40   47   44   20
  52   50   94   81   56   78   26
  60    8   10   68   57   48   14
  82   73   58   85   87   32   65
  22   65   37    9   20    2   81
  34   26   39   24   96   53   44
  54   19   64   26   63   19   11
  95   17   56   93   88   83   10
[torch.LongTensor of size 10x7]

   0    0    0    0    0    0    0
   0    0    0    0    0    0    0
   3   53  100   40   47   44   20
   0    0    0    0    0    0    0
  60    8   10   68   57   48   14
  82   73   58   85   87   32   65
  22   65   37    9   20    2   81
  34   26   39   24   96   53   44
  54   19   64   26   63   19   11
  95   17   56   93   88   83   10
[torch.DoubleTensor of size 10x7]



In [8]:
totalPredsummary = torch.zeros(n, n_s * k)

for i=1, n_s do 
    preds = model:forward({sentences[i], queries, torch.zeros(n, q)})
    -- Pulling the best actions
    qMax, qindx = torch.max(preds, 2)

    -- Here's the fast way to select the optimal action for each query
    actions = torch.zeros(n, 2):scatter(2, qindx, torch.ones(preds:size()))
    predsummary = buildPredsummaryFast(predsummary, actions, sentences[i], SELECT)
    buildTotalSummaryFast(predsummary, totalPredsummary)
--     buildTotalSummary(predsummary, totalPredsummary)
end

print("predictions = ")
print(preds)

predictions = 	
-0.1101 -0.2236
 0.0988 -0.0044
-0.1624  0.0977
 0.4590 -0.0133
-0.2038  0.1108
-0.1785  0.1961
-0.2048  0.1665
-0.1223  0.3094
-0.2750  0.1111
-0.2313  0.1707
[torch.DoubleTensor of size 10x2]



In [ ]:
function stackMemory(newinput, memory_hist, memsize, adapt, use_cuda)
    local sentMemory = torch.cat(newinput[1][1]:double(), memory_hist[1][1]:double(), 1)
    local queryMemory = torch.cat(newinput[1][2]:double(), memory_hist[1][2]:double(), 1)
    local sumryMemory = torch.cat(newinput[1][3]:double(), memory_hist[1][3]:double(), 1)
    local rewardMemory = torch.cat(newinput[2]:double(), memory_hist[2]:double(), 1)

    if adapt then
        regMemory = torch.cat(newinput[4]:double(), memory_hist[4]:double(), 1)
    end 

    if use_cuda then 
        actionMemory = torch.cat(newinput[3]:double(), memory_hist[3]:double(), 1)
    else 
        actionMemory = torch.cat(newinput[3], memory_hist[3], 1)
    end
    --- specifying rows to index 
    if sentMemory:size(1) <= memsize then
        nend = sentMemory:size(1)
        nstart = 1
    else 
        nstart = math.max(memsize - sentMemory:size(1), 1)
        nend = memsize + nstart
    end
    --- Selecting n last data points
    sentMemory = sentMemory[{{nstart, nend}}]
    queryMemory = queryMemory[{{nstart, nend}}]
    sumryMemory = sumryMemory[{{nstart, nend}}]
    rewardMemory = rewardMemory[{{nstart, nend}}]
    actionMemory = actionMemory[{{nstart, nend}}]

    if use_cuda then
        inputMemory = {sentMemory:cuda(), queryMemory:cuda(), sumryMemory:cuda()}
        rewardMemory = rewardMemory:cuda()
        actionMemory = torch.ByteTensor(#actionMemory):copy(actionMemory):cuda()
    end

    inputMemory = {sentMemory, queryMemory, sumryMemory}
    if adapt then
        regMemory = regMemory[{{nstart, nend}}]
        return {inputMemory, rewardMemory, actionMemory, regMemory}
    end 
    return {inputMemory, rewardMemory, actionMemory}
end    